In [1]:
import cv2
import numpy as np
import pylab
import os
import shutil
import imageio

In [2]:
def process_image(im):
    image, contours, hier = cv2.findContours(im, cv2.RETR_TREE,
                    cv2.CHAIN_APPROX_SIMPLE)

    # with each contour, draw boundingRect in green
    # a minAreaRect in red and
    # a minEnclosingCircle in blue
    for c in contours:
        
        # get the bounding rect
        x, y, w, h = cv2.boundingRect(c)
        
        # draw a green rectangle to visualize the bounding rect
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # get the min area rect
        rect = cv2.minAreaRect(c)
        box = cv2.boxPoints(rect)
        
        # convert all coordinates floating point values to int
        box = np.int0(box)
        
        # draw a red 'nghien' rectangle
        cv2.drawContours(img, [box], 0, (0, 0, 255))

    cv2.drawContours(img, contours, -1, (255, 255, 0), 1)
    return img


In [3]:
try:
    os.mkdir("images")
except:
    shutil.rmtree("images")
    os.mkdir("images")

In [4]:
try:
    %rm -rf foo.avi
except:
    pass

In [5]:
filename = 'vtest.mp4'
vid = imageio.get_reader(filename,  'ffmpeg')

for num in range(vid.get_length()):
    img = vid.get_data(num)
    gray_image = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    _,im =  cv2.threshold(gray_image,220,1,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    im = process_image(im)
    pylab.imsave("images/"+"image-"+str(num)+".jpg",im)

In [6]:
!ffmpeg -framerate 25 -i images/image-%00d.jpg -r 76 -s 800x600 foo_1.mp4

ffmpeg version 2.7 Copyright (c) 2000-2015 the FFmpeg developers
  built with llvm-gcc 4.2.1 (LLVM build 2336.11.00)
  configuration: --prefix=/Volumes/Ramdisk/sw --enable-gpl --enable-pthreads --enable-version3 --enable-libspeex --enable-libvpx --disable-decoder=libvpx --enable-libmp3lame --enable-libtheora --enable-libvorbis --enable-libx264 --enable-avfilter --enable-libopencore_amrwb --enable-libopencore_amrnb --enable-filters --enable-libgsm --enable-libvidstab --enable-libx265 --disable-doc --arch=x86_64 --enable-runtime-cpudetect
  libavutil      54. 27.100 / 54. 27.100
  libavcodec     56. 41.100 / 56. 41.100
  libavformat    56. 36.100 / 56. 36.100
  libavdevice    56.  4.100 / 56.  4.100
  libavfilter     5. 16.101 /  5. 16.101
  libswscale      3.  1.101 /  3.  1.101
  libswresample   1.  2.100 /  1.  2.100
  libpostproc    53.  3.100 / 53.  3.100
[mjpeg @ 0x7f80d7001600] bits 131 is invalid
[mjpeg @ 0x7f80d7001600] mjpeg: unsupported coding type (cd)
    Last message repeat

In [ ]:
# 